In [83]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Inflation

In [84]:
df_infl = pd.read_csv("Inflation.csv", sep = ";")
df_infl.Date = pd.to_datetime(df_infl.Date, dayfirst = True)

print(df_infl)

          Date CH_infl       US_infl
0   2005-01-01    -0,5    0,21019443
1   2005-02-01     0,2   0,576822234
2   2005-03-01     0,2   0,782064651
3   2005-04-01     0,8   0,672529747
4   2005-05-01    -0,1  -0,102774923
..         ...     ...           ...
199 2021-08-01     0,2   0,032600374
200 2021-09-01     0,0   0,410850556
201 2021-10-01     0,3   0,866476548
202 2021-11-01     0,0   0,699229907
203 2021-12-01    -0,1    0,57517485

[204 rows x 3 columns]


### Short term interest rates

In [85]:
df_IR = pd.read_csv("ShortTerm.csv", sep = ";")
df_IR.Date = pd.to_datetime(df_IR.Date, dayfirst = True)
df_IR = df_IR.dropna(axis=0, how='all')
df_IR = df_IR.fillna(method="pad", limit=1)


print(df_IR)
print(f'Nb Nan: {df_IR.US_IR.isnull().sum()}')

           Date SARON_close US_IR
0    2005-01-03    0,588191  1,99
1    2005-01-04    0,616042  2,05
2    2005-01-05    0,696835  2,04
3    2005-01-06     0,76842  2,04
4    2005-01-07    0,672936  2,03
...         ...         ...   ...
4430 2021-12-27   -0,703335  0,04
4431 2021-12-28   -0,703774  0,03
4432 2021-12-29   -0,703104  0,01
4433 2021-12-30   -0,685636  0,06
4434 2021-12-31    -0,68483  0,06

[4435 rows x 3 columns]
Nb Nan: 0


### Exchange rates

In [86]:
df = pd.read_csv("CHF=X.csv")

#data cleaning & processing 
df.Date = pd.to_datetime(df.Date)
df = df.fillna(method="pad", limit=1)

print(df)

           Date      Open     High      Low     Close  Adj Close  Volume
0    2005-01-03  1.137800  1.15500  1.13740  1.147000   1.147000     0.0
1    2005-01-04  1.147100  1.17060  1.14430  1.167600   1.167600     0.0
2    2005-01-05  1.167600  1.17540  1.16550  1.166900   1.166900     0.0
3    2005-01-06  1.167200  1.17730  1.16670  1.174000   1.174000     0.0
4    2005-01-07  1.174000  1.18850  1.16760  1.184100   1.184100     0.0
...         ...       ...      ...      ...       ...        ...     ...
4430 2021-12-27  0.918774  0.91993  0.91710  0.918781   0.918781     0.0
4431 2021-12-28  0.916990  0.91888  0.91580  0.917160   0.917160     0.0
4432 2021-12-29  0.916900  0.91992  0.91265  0.916900   0.916900     0.0
4433 2021-12-30  0.913900  0.91804  0.91300  0.913600   0.913600     0.0
4434 2021-12-31  0.913900  0.91474  0.91047  0.913700   0.913700     0.0

[4435 rows x 7 columns]


### Concatenate

In [87]:
df_fin = pd.concat([df,df_IR.drop(labels = 'Date', axis = 1)], axis = 1)

print(df_fin)


           Date      Open     High      Low     Close  Adj Close  Volume  \
0    2005-01-03  1.137800  1.15500  1.13740  1.147000   1.147000     0.0   
1    2005-01-04  1.147100  1.17060  1.14430  1.167600   1.167600     0.0   
2    2005-01-05  1.167600  1.17540  1.16550  1.166900   1.166900     0.0   
3    2005-01-06  1.167200  1.17730  1.16670  1.174000   1.174000     0.0   
4    2005-01-07  1.174000  1.18850  1.16760  1.184100   1.184100     0.0   
...         ...       ...      ...      ...       ...        ...     ...   
4430 2021-12-27  0.918774  0.91993  0.91710  0.918781   0.918781     0.0   
4431 2021-12-28  0.916990  0.91888  0.91580  0.917160   0.917160     0.0   
4432 2021-12-29  0.916900  0.91992  0.91265  0.916900   0.916900     0.0   
4433 2021-12-30  0.913900  0.91804  0.91300  0.913600   0.913600     0.0   
4434 2021-12-31  0.913900  0.91474  0.91047  0.913700   0.913700     0.0   

     SARON_close US_IR  
0       0,588191  1,99  
1       0,616042  2,05  
2       0,69

In [102]:
# Add columns
df_fin["CH_infl"] = np.nan
df_fin["US_infl"] = np.nan

cur = 0
cur_month = [df_infl.Date[cur].month, df_infl.Date[cur].year]
for i, date in enumerate(df_fin.Date):
    if [date.month, date.year] == cur_month:
        df_fin.CH_infl[i] = df_infl.CH_infl[cur]
        df_fin.US_infl[i] = df_infl.US_infl[cur]
        cur = cur+1
        if cur >= df_infl.shape[0]:
            break
        cur_month = [df_infl.Date[cur].month, df_infl.Date[cur].year]
    

df_fin = df_fin.fillna(method="ffill")
print(df_fin.iloc[:40,:])


         Date    Open    High     Low   Close  Adj Close  Volume SARON_close  \
0  2005-01-03  1.1378  1.1550  1.1374  1.1470     1.1470     0.0    0,588191   
1  2005-01-04  1.1471  1.1706  1.1443  1.1676     1.1676     0.0    0,616042   
2  2005-01-05  1.1676  1.1754  1.1655  1.1669     1.1669     0.0    0,696835   
3  2005-01-06  1.1672  1.1773  1.1667  1.1740     1.1740     0.0     0,76842   
4  2005-01-07  1.1740  1.1885  1.1676  1.1841     1.1841     0.0    0,672936   
5  2005-01-10  1.1837  1.1848  1.1785  1.1794     1.1794     0.0    0,674866   
6  2005-01-11  1.1792  1.1837  1.1733  1.1824     1.1824     0.0    0,648851   
7  2005-01-12  1.1825  1.1840  1.1629  1.1676     1.1676     0.0    0,733961   
8  2005-01-13  1.1676  1.1739  1.1665  1.1721     1.1721     0.0    0,738232   
9  2005-01-14  1.1720  1.1865  1.1720  1.1791     1.1791     0.0    0,694707   
10 2005-01-17  1.1813  1.1848  1.1770  1.1845     1.1845     0.0     0,59171   
11 2005-01-18  1.1845  1.1902  1.1806  1

C:\Users\franc\AppData\Local\Temp\ipykernel_17284\1291184814.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.CH_infl[i] = df_infl.CH_infl[cur]
C:\Users\franc\AppData\Local\Temp\ipykernel_17284\1291184814.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.US_infl[i] = df_infl.US_infl[cur]


In [105]:
df_fin.to_csv("dataset.csv", index = False)